> **GPT-2와 같은 언어 모델을 미세 조정하는 과정**


1. Causal Language Modeling (CLM) 미세 조정:

  모델 체크포인트로 'distilgpt2'를 사용하여 언어 모델을 미세 조정합니다.
Trainer 클래스를 사용하여 학습 데이터를 기반으로 모델을 학습(trainer.train())시킵니다.

2. Masked Language Modeling (MLM) 미세 조정:

  'distilroberta-base' 모델을 사용하여 마스킹된 언어 모델(Masked Language Model)을 미세 조정합니다.
Trainer 클래스와 DataCollatorForLanguageModeling을 사용하여 마스킹 확률을 설정하고 학습을 진행합니다.

#목차
사전 준비:

*   datasets와 transformers 라이브러리 설치.

*   Hugging Face에 로그인하고 Git LFS 설치.




데이터 준비:

*   wikitext-2-raw-v1 데이터셋을 로드하고 일부 데이터를 미리보기.

*  show_random_elements() 함수로 랜덤 데이터를 출력.

*   데이터를 토큰화하고 이를 모델 학습에 적합하게 변환.



Causal Language Modeling(CLM):

*   GPT-2 모델을 불러와서 학습. 이 경우 'distilgpt2'를 사용.

*   텍스트 데이터를 토큰화한 후 특정 길이 (block_size)로 나눔.

*   Trainer 클래스를 이용해 모델 학습.


Masked Language Modeling(MLM):


*   distilroberta-base 모델 사용.
*   마스크드 언어 모델(Masked Language Model)은 일부 토큰을 무작위로 마스킹하고 이를 예측하는 방식으로 학습.

*   DataCollatorForLanguageModeling을 사용하여 각 에폭마다 무작위 마스킹을 보장.












#사전준비과정

In [ ]:
! pip install datasets transformers

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.9 MB/s eta 0:00:00


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
!apt install git-lfs

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [ ]:
import transformers

print(transformers.__version__)

4.44.2


In [ ]:
from transformers.utils import send_example_telemetry

send_example_telemetry("language_modeling_notebook", framework="pytorch")

# Fine-tuning a language model


#학습 결과 미리보기
![Widget inference representing the causal language modeling task](https://github.com/huggingface/notebooks/blob/main/examples/images/causal_language_modeling.png?raw=1)




## Preparing the dataset

 [Wikitext 2]()을 데이터 셋으로 사용한다.

In [ ]:
from datasets import load_dataset
datasets = load_dataset('wikitext', 'wikitext-2-raw-v1')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/733k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/6.36M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/657k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/4358 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/36718 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

load_dataset: Hugging Face 데이터셋 라이브러리에서 "wikitext-2-raw-v1" 데이터셋 로드

To access an actual element, you need to select a split first, then give an index:

In [ ]:
datasets["train"][10]

{'text': ' The game \'s battle system , the BliTZ system , is carried over directly from Valkyira Chronicles . During missions , players select each unit using a top @-@ down perspective of the battlefield map : once a character is selected , the player moves the character around the battlefield in third @-@ person . A character can only act once per @-@ turn , but characters can be granted multiple turns at the expense of other characters \' turns . Each character has a field and distance of movement limited by their Action Gauge . Up to nine characters can be assigned to a single mission . During gameplay , characters will call out if something happens to them , such as their health points ( HP ) getting low or being knocked out by enemy attacks . Each character has specific " Potentials " , skills unique to each character . They are divided into " Personal Potential " , which are innate skills that remain unaltered unless otherwise dictated by the story and can either help or impede

show_random_element()를 통해 데이터의 모양을 파악해볼 수 있습니다.

In [ ]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(datasets["train"])

,text
0,"In the days leading up to the Peach Bowl , one sportswriter called Virginia Tech 's 1986 football season a "" season of surprises . "" Tech began the season having gone 6 – 5 in 1985 . In their first game , the Hokies faced the Cincinnati Bearcats . Tech lost , 24 – 20 , on a last @-@ minute play that saw a Cincinnati pass tipped twice and caught for a sustaining first down . The drive eventually resulted in a game @-@ winning touchdown for Cincinnati . \n"
1,
2,"Still , many men struggle to find acceptance within the role of stay @-@ at @-@ home dad despite the many gains that have been made . Many worry about losing business skills and their "" professional place in line "" . There is a common misconception that stay @-@ at @-@ home dads cannot get a job and therefore must rewrite the typical family roles , forcing the wife into the workforce . Carrying the financial burden and dealing with children 's attachment to the dad can be difficult on a working mother . \n"
3,"Foliot spent most of his tenure of office in his diocese , only rarely attending the royal court or being assigned governmental duties . On 30 December 1223 , Foliot assumed one of those duties , when he took custody of Hereford Castle after it was surrendered by Hubert de Burgh , during the redistribution of royal castles when de Burgh ousted des Roches from power . He also was appointed to determine the size of the royal forest in Gloucestershire . Foliot also founded a hospital in Ledbury , devoted to St Katherine . He helped found Alberbury Priory , a house of the Grandmontine order . In his cathedral , he reorganised the benefices and offices of the chapter , as well as endowing further benefices . \n"
4,"Weir 's health had not completely recovered by the time he embarked for Egypt , and he rejoined his battalion on 4 March 1916 . After his departure , the 10th had fought through the remainder of the campaign before being withdrawn along with the rest of the Allied force in December 1915 . The battalion was subsequently moved back to Egypt . In mid @-@ 1916 , the bulk of the AIF was transferred to the Western Front , and Weir led the 10th Battalion through July and August 1916 during the Battles of Pozières and Mouquet Farm . At Pozières , the battalion suffered 350 casualties in four days . By the time of the battle , Weir was the only original battalion commander remaining in the 1st Australian Division , and had turned 50 years of age . On 23 August , immediately after Mouquet Farm , Weir was again appointed acting commander of the 3rd Brigade . Exhausted , on 7 September 1916 he asked to be relieved , and his request was granted . He returned to Australia on 23 September 1916 , and his AIF appointment was terminated on 14 December . In the Australian official history of the war , Bean observed that despite his age , Weir "" took his battalion into the front line , commanded it there throughout its first battle , and remained longer in the field than almost any of the senior militia officers who had left with the original force "" . \n"
5,"The Qedarites are among a number of North Arabian tribes whose interactions with Arameaen tribes beginning in the 8th century BCE resulted in cultural exchanges between these two large Semitic groups . Early Arab tribal groups like the Qedarites spoke early Arab dialects , but as the Arabic alphabet had not yet been developed , they used the Aramaic alphabet to write . "" The tongue of Kedar "" is used in rabbinical sources as a name for the Arabic language . \n"
6,= = Background = = \n
7,"Aston Villa Football Club were formed in March 1874 , by members of the Villa Cross Wesleyan Chapel in Handsworth which is now part of Birmingham . The four founders of Aston Villa were Jack Hughes , Frederick Matthews , Walter Price and William Scattergood . Aston Villa 's first match was against the local Aston Brook St Mary 's Rugby team . As a condition of the match , the Villa side had to agree to play the first half 

보시다시피 일부 텍스트는 위키백과 기사의 전체 단락인 반면, 일부 텍스트는 제목이나 빈 줄에 불과합니다.

## Causal Language modeling

In [ ]:
model_checkpoint = "distilgpt2"

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"]) # 텍스트를 토큰으로 변환

 'datasets' 개체의 모든 분할에 적용하여 'batched=True'와 4개의 프로세스를 사용하여 전처리 속도를 높입니다. 나중에 '텍스트' 열이 필요하지 않으므로 이 열을 버립니다.

In [ ]:
tokenized_datasets = datasets.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])

Map (num_proc=4):   0%|          | 0/4358 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/36718 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/3760 [00:00<?, ? examples/s]

map: 데이터셋의 각 예시에 tokenize_function을 적용


batched=True: 배치 처리를 통해 빠르게 토큰화


remove_columns=["text"]: 원본 텍스트 열을 제거하고 토큰화된 데이터만 남김

토큰 인덱스 확인

In [ ]:
tokenized_datasets["train"][1]

{'input_ids': [796, 569, 18354, 7496, 17740, 6711, 796, 220, 198],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}

모든 텍스트를 서로 연결한 다음 결과를 특정 'block_size'의 작은 덩어리로 나누어야 한다는 것입니다.

먼저, 모델이 사전 학습된 최대 길이를 가져옵니다. GPU RAM에 넣기에는 너무 클 수 있으므로 128로 조금 줄였습니다.

In [ ]:
block_size = 128 # 학습에 사용할 시퀀스 길이 /#block_size: 한 번에 모델에 입력할 텍스트 길이


In [ ]:
def group_texts(examples):
    concatenated = {k: sum(examples[k], []) for k in examples.keys()}  # 예시들을 하나의 시퀀스로 합침
    total_length = len(concatenated["input_ids"])  # 전체 토큰 길이 계산
    total_length = (total_length // block_size) * block_size  # block_size에 맞춰 자르기
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]  # block_size 단위로 나누기
        for k, t in concatenated.items()
    }
    return result

기본적으로 '지도' 방법은 전처리 함수로 처리할 1,000개의 예제 배치를 전송합니다.

 따라서 나머지는 연결된 토큰화된 텍스트를 1,000개의 예제마다 'block_size'의 배수로 만들기 위해 여기서는 삭제합니다.

 이 동작은 더 높은 배치 크기를 전달하여 조정할 수 있습니다(더 느리게 처리됨).

 다중 처리를 사용하여 전처리 속도를 높일 수도 있습니다:

In [ ]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)
#group_texts: 토큰들을 block_size에 맞춰 잘라 모델에 입력할 형태로 만듦

Map (num_proc=4):   0%|          | 0/4358 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/36718 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/3760 [00:00<?, ? examples/s]

In [ ]:
tokenizer.decode(lm_datasets["train"][1]["input_ids"])

' game and follows the " Nameless ", a penal military unit serving the nation of Gallia during the Second Europan War who perform secret black operations and are pitted against the Imperial unit " Calamaty Raven ". \n The game began development in 2010, carrying over a large portion of the work done on Valkyria Chronicles II. While it retained the standard features of the series, it also underwent multiple adjustments, such as making the game more forgiving for series newcomers. Character designer Raita Honjou and composer Hitoshi Sakimoto both returned from previous entries, along with Valkyria Chronicles II director Takeshi Oz'

학습할 데이터가 준비되었고, 이제 이 데이터 기반으로 trainer를 생성할 준비가 되었습니다.

In [ ]:
from transformers import AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained(model_checkpoint)

model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

And some `TrainingArguments`:

In [ ]:
from transformers import Trainer, TrainingArguments

In [ ]:
model_name = model_checkpoint.split("/")[-1]
training_args = TrainingArguments(
    f"{model_name}-finetuned-wikitext2", # 학습 결과를 저장할 디렉토리
    evaluation_strategy = "epoch", # 매 에폭마다 평가
    learning_rate=2e-5,# 학습률 설정
    weight_decay=0.01,# 가중치 감쇠 설정
    push_to_hub=True,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


We pass along all of those to the `Trainer` class:

In [ ]:
trainer = Trainer(
    model=model,  # 학습시킬 모델 (GPT-2 등 사전 학습된 언어 모델)
    args=training_args,  # 학습 관련 하이퍼파라미터 및 설정 (학습률, 에폭 수 등)
    train_dataset=lm_datasets["train"],  # 모델을 학습시킬 훈련 데이터셋
    eval_dataset=lm_datasets["validation"],  # 학습 중 모델을 평가할 검증 데이터셋
)


우리 모델을 실행시킨다

In [ ]:
trainer.train()

ValueError: The model did not return a loss from the inputs, only the following keys: logits,past_key_values. For reference, the inputs it received are input_ids,attention_mask.

모델평가

In [ ]:
import math

# 모델을 검증 데이터셋을 사용해 평가하고 결과를 반환
eval_results = trainer.evaluate()

# 평가 결과에서 손실값('eval_loss')을 가져와서 Perplexity(언어 모델의 정확도를 나타내는 지표)를 계산
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")


AttributeError: 'NotebookTrainingTracker' object has no attribute 'value'

You can now upload the result of the training to the Hub, just execute this instruction:

In [ ]:
trainer.push_to_hub()

model.safetensors:   0%|          | 0.00/328M [00:00<?, ?B/s]

events.out.tfevents.1727677311.9d0bfecd729d.1570.1:   0%|          | 0.00/359 [00:00<?, ?B/s]

events.out.tfevents.1727675548.9d0bfecd729d.1570.0:   0%|          | 0.00/9.41k [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/0208suin/distilgpt2-finetuned-wikitext2/commit/17de40a2d33bdffadf356828eaf5951a8a9bc363', commit_message='End of training', commit_description='', oid='17de40a2d33bdffadf356828eaf5951a8a9bc363', pr_url=None, pr_revision=None, pr_num=None)

You can now share this model with all your friends, family, favorite pets: they can all load it with the identifier `"your-username/the-name-you-picked"` so for instance:

```python
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained("sgugger/my-awesome-model")
```

## Masked language modeling

일부 토큰은 무작위로 마스킹하고([MASK]로 대체하여) 라벨은 마스킹된 토큰만 포함하도록 조정됩니다(마스킹되지 않은 토큰은 예측할 필요가 없습니다).


We will use the [`distilroberta-base`](https://huggingface.co/distilroberta-base) model for this example. You can pick any of the checkpoints listed [here](https://huggingface.co/models?filter=masked-lm) instead:

In [ ]:
model_checkpoint = "distilroberta-base"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)  # 사전 학습된 토크나이저 로드, 'use_fast=True'로 빠른 토크나이저 사용

# 데이터셋의 각 예시에 대해 토크나이즈 함수를 적용하여 토큰화된 데이터셋 생성
tokenized_datasets = datasets.map(
    tokenize_function,  # 토크나이즈 함수를 적용
    batched=True,  # 배치 단위로 처리하여 속도 향상
    num_proc=4,  # 병렬 처리할 프로세스 수 (4개)
    remove_columns=["text"]  # 토큰화 이후 원본 'text' 열 제거
)


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

Map (num_proc=4):   0%|          | 0/4358 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (544 > 512). Running this sequence through the model will result in indexing errors


Map (num_proc=4):   0%|          | 0/36718 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (560 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (528 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (638 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (522 > 512). Running this sequence through the model will result in indexing errors


Map (num_proc=4):   0%|          | 0/3760 [00:00<?, ? examples/s]

나머지는 두 가지 예외를 제외하고는 우리가 가지고 있는 것과 매우 유사합니다.

먼저 마스킹된 LM에 적합한 모델을 사용합니다:

In [ ]:
from transformers import AutoModelForMaskedLM

# 사전 학습된 마스크드 언어 모델을 로드
model = AutoModelForMaskedLM.from_pretrained(model_checkpoint)


model.safetensors:   0%|          | 0.00/331M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


재정의한다 our `TrainingArguments`:

In [ ]:
model_name = model_checkpoint.split("/")[-1]  # 모델 체크포인트에서 마지막 부분만 가져와 모델 이름을 추출

training_args = TrainingArguments(
    f"{model_name}-finetuned-wikitext2",  # 학습 결과를 저장할 디렉토리 이름
    evaluation_strategy="epoch",  # 매 에폭마다 모델을 평가
    learning_rate=2e-5,  # 학습률 설정
    weight_decay=0.01,  # 가중치 감쇠 적용
    push_to_hub=True,  # Hugging Face Hub에 학습된 모델을 푸시할지 여부 설정
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


마지막으로 특별한 '데이터_콜라터'를 사용합니다. '데이터_콜라터'는 샘플을 채취하여 텐서로 배치하는 기능입니다. 이전 예제에서는 특별히 할 일이 없었기 때문에 이 인수에 기본값을 사용했습니다. 여기서는 무작위 마스킹을 수행하고자 합니다. 토큰화와 같은 전처리 단계로 수행할 수 있지만 토큰은 각 에포크에서 항상 동일한 방식으로 마스킹됩니다. '데이터_콜라터' 내부에서 이 단계를 수행함으로써 데이터를 검토할 때마다 이 무작위 마스킹이 새로운 방식으로 수행되도록 보장합니다.

이 마스킹을 위해 라이브러리에서 '언어 모델링을 위한 데이터 콜렉터'를 제공합니다. 마스킹 확률을 조정할 수 있습니다:

In [ ]:
from transformers import DataCollatorForLanguageModeling
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

KeyboardInterrupt: 

그런 다음 모든 것을 '트레이너'에게 전달하고 훈련을 시작하기만 하면 됩니다

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["validation"],
    data_collator=data_collator,
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,2.081700,1.934387
2,1.985600,1.883658
3,1.943400,1.835446


TrainOutput(global_step=7218, training_loss=2.0313560378026287, metrics={'train_runtime': 1889.1984, 'train_samples_per_second': 30.556, 'train_steps_per_second': 3.821, 'total_flos': 1913931601832448.0, 'train_loss': 2.0313560378026287, 'epoch': 3.0})

 MLM 목표의 경우 나머지 토큰에 액세스하면서 마스킹된 토큰(전체 토큰의 15%를 차지)에 대한 예측만 하면 되기 때문에 CLM 목표보다 당혹감이 훨씬 낮습니다. 따라서 모델에 더 쉬운 작업입니다.

In [ ]:
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")